In [2]:
import hyperspy.api as hs
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [3]:
import maskselector_gui as msg

In [3]:
#To be activated if you want to plot outside of the notebook
%pylab qt

Populating the interactive namespace from numpy and matplotlib


### Loading data

In [4]:
haadf = hs.load('46_data/45_haadf_20k_as_44.emi')

In [5]:
adf = hs.load('46_data/44_adf.dm3')

In [85]:
ebic = hs.load('46_data/44_ebic_area_13_20k.dm3')

In [7]:
Tecnai_ebic = hs.load('46_data/21 ebic as 20 better_tecnai.dm3')

In [8]:
edx = hs.load('46_data/46_edx_20k_as_44.rpl')

Loading as spectrum


In [9]:
edx.axes_manager[2].units = "eV"
edx.axes_manager[2].scale = 9.952
edx.axes_manager[2].offset = -470

In [93]:
edx.plot(navigator=ebic)

In [94]:
ebic.axes_manager.gui()

In [92]:
Tecnai_ebic.plot()

### Normalization of the EBIC map in nA units (ampli. 0.2nA/V)

In [50]:
vv_100mV = hs.load("41_ebic_100mV_p-p.dm3")

In [87]:
ebic.change_dtype("float64")

In [88]:
ebic.data[:,:] = (ebic.data[:,:]-amin(ebic.data[:,:]))*0.2/(amax(vv_100mV.data)-amin(vv_100mV.data))

In [89]:
edx.plot(navigator=ebic)

### Analysis of EDX data

In [ ]:
#mask_obj = msg.Interactive_plot(ebic.data)

In [ ]:
#mask_vac = mask_obj.main.mask

In [ ]:
#save("mask", mask_vac)

In [10]:
mask_vac = load("mask.npy")

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
im = ax1.imshow(mask_vac[:,:])

In [11]:
ave_edx_ebic_true = np.zeros_like(edx.data[0,0,:], dtype=uint64)
ave_edx_ebic_false = np.zeros_like(edx.data[0,0,:], dtype=uint64)
true = 0
false = 0

for i in range(len(edx.data[:,0,0])):
    for j in range(len(edx.data[0,:,0])):
        if mask_vac[i,j] == True:
            ave_edx_ebic_true[:] = ave_edx_ebic_true[:] + edx.data[i,j,:]
            true = true + 1
        else:
            ave_edx_ebic_false[:] = ave_edx_ebic_false[:] + edx.data[i,j,:]
            false = false + 1

In [12]:
x_ev = np.zeros_like(edx.data[0,0,:], dtype=float16)
for i in range(len(x_ev)):
    x_ev[i]=i*9.952-470

#### Plot of the averaged spectra inside and outside the mask

In [17]:
plot(x_ev, float64(ave_edx_ebic_true)/float64(true), label='inside', lw=2)
plot(x_ev, float64(ave_edx_ebic_false)/float64(false)-0.025, label='outside', lw=2)
plot(x_ev, float64(ave_edx_ebic_true)/float64(true)-float64(ave_edx_ebic_false)/float64(false)-0.03, label='difference', lw=2)
pylab.ylim([0,1])
plt.axvspan(240, 320, color="green", alpha=0.5)
#plt.axvline(x=280, label='C') #C
#plt.axvline(x=390) #N
plt.axvspan(350, 430, color="red", alpha=0.5)
#plt.axvline(x=520) #O
plt.axvspan(480, 560, color="blue", alpha=0.5)
#plt.axvline(x=1012) #Zn La
#plt.axvline(x=1486) #Al Ka
#plt.axvline(x=1739) #Si K
#plt.axvline(x=1774) #W M
#plt.axvline(2290, 2390, color="orange", alpha=0.5) #Pb M
plt.axvspan(3900, 3980, color="yellow", alpha=0.5)
#plt.axvline(x=3937) #I La

In [33]:
fig, ax = plt.subplots(1, 1, figsize=[24, 13])

plot(x_ev, float64(ave_edx_ebic_true)/float64(true), lw=4) #, cmap='gray'
plot(x_ev, float64(ave_edx_ebic_false)/float64(false)-0.025, lw=4)
plot(x_ev, float64(ave_edx_ebic_true)/float64(true)-float64(ave_edx_ebic_false)/float64(false)-0.03, lw=4)
pylab.xlim([130,5000])
plt.axvspan(240, 320, color="green", alpha=0.5)
plt.axvspan(350, 430, color="red", alpha=0.5)
plt.axvspan(480, 560, color="blue", alpha=0.5)
plt.axvspan(2300, 2380, color="orange", alpha=0.5) #Pb M
plt.axvspan(3900, 3980, color="yellow", alpha=0.5)
plt.xlabel('eV', fontsize=25)
plt.xticks(fontsize=25)
plt.ylabel('Average counts per pixel', fontsize=25)
plt.yticks(fontsize=25)

axins = inset_axes(ax,
                   width="50%",  # width = 30% of parent_bbox
                   height=2.5,  # height : 1 inch
                   loc=1)
im = plot(x_ev, float64(ave_edx_ebic_true)/float64(true), lw=3) #, cmap='gray'
im = plot(x_ev, float64(ave_edx_ebic_false)/float64(false)-0.025, label='outside', lw=3)
pylab.xlim([130,20000])
plt.yticks(visible=False)
plt.xlabel('eV', fontsize=15)
plt.xticks(fontsize=15)

plt.draw()
plt.show()
fig.savefig('46_image/Spectra.png', dpi=300)

#### Plot of the maps

In [91]:
C_map = edx.integrate_in_range(signal_range=(240.,320.))
C_map_bin = C_map.rebin((256,256))

/home/martial/hyperspy09/src/hyperspy/hyperspy/signal.py:2622: VisibleDeprecationWarning: Indexing the `Signal` class is deprecated and will be removed in HyperSpy 0.9. Please use `.isig` and/or `.inav` instead.
  VisibleDeprecationWarning)


In [73]:
C_map_bin.plot()

In [19]:
N_map = edx.integrate_in_range(signal_range=(350.,430.))
N_map_bin = N_map.rebin((256,256))

In [ ]:
N_map_bin.plot()

In [20]:
O_map = edx.integrate_in_range(signal_range=(480.,560.))
O_map_bin = O_map.rebin((256,256))

In [ ]:
O_map_bin.plot()

In [27]:
Pb_map = edx.integrate_in_range(signal_range=(2290.,2390.))
Pb_map_bin = Pb_map.rebin((256,256))

In [22]:
I_map = edx.integrate_in_range(signal_range=(3900.,3980.))
I_map_bin = I_map.rebin((256,256))

In [ ]:
I_map_bin.plot()

### Plot data

In [29]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.01, wspace=0.)

fig.set_size_inches(6.2,5.915)
#fig.set_dpi(80)
#ax.set_title('Average')
#plt.text(1.5*specImg3D.data.shape[1]-r1+20, specImg3D.data.shape[2]/4, "1 A", fontsize=50)
#plt.text(1.5*specImg3D.data.shape[1]-r2+20, specImg3D.data.shape[2]/6, "0.8 A", fontsize=50)

ax1 = fig.add_subplot(331)
im = ax1.imshow(Tecnai_ebic.data[:,:]) #, cmap='gray'
plt.text(12.5, Tecnai_ebic.data.shape[1]-25, '(a)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

ax2 = fig.add_subplot(332)
im = ax2.imshow(haadf.data[:,:]) #, cmap='gray'
plt.text(50, haadf.data.shape[1]-100, '(b)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

ax3 = fig.add_subplot(333)
im = ax3.imshow(adf.data[:,:]) #, cmap='gray'
plt.text(25, adf.data.shape[1]-50, '(c)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

ax4 = fig.add_subplot(334)
im = ax4.imshow(C_map_bin.data[:,:]) #, cmap='gray'
plt.text(12.5/2, I_map_bin.data.shape[1]-12.5, '(d)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

ax5 = fig.add_subplot(335)
im = ax5.imshow(N_map_bin.data[:,:]) #, cmap='gray'
plt.text(12.5/2, N_map_bin.data.shape[1]-12.5, '(e)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

ax6 = fig.add_subplot(336)
im = ax6.imshow(O_map_bin.data[:,:]) #, cmap='gray'
plt.text(12.5/2, O_map_bin.data.shape[1]-12.5, '(f)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

ax8 = fig.add_subplot(337)
im = ax8.imshow(Pb_map_bin.data[:,:]) #, cmap='gray'
plt.text(12.5/2, Pb_map_bin.data.shape[1]-12.5, '(g)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

ax7 = fig.add_subplot(338)
im = ax7.imshow(I_map_bin.data[:,:]) #, cmap='gray'
plt.text(12.5/2, I_map_bin.data.shape[1]-12.5, '(h)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

ax9 = fig.add_subplot(339)
im = ax9.imshow(ebic.data[:,:]) #, cmap='gray'
plt.text(25, ebic.data.shape[1]-50, '(i)', fontsize=12, bbox=dict(facecolor='white', alpha=1))
plt.axis('off')

fig.savefig('46_image/fig1.png', dpi=300)